<a href="https://colab.research.google.com/github/fenzhantw/DLforeveryone/blob/main/DeepLearning_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import tensorflow.contrib.eager as tfe
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
pip uninstall tensorflow

In [ ]:
pip install tensorflow==1.13.2

In [ ]:
tf.enable_eager_execution()
tf.set_random_seed(777)
print(tf.__version__)

1.13.2


In [ ]:
X_data = [[0,0],
          [0,1],
          [1,0],
          [1,1]]
y_data = [[0],
          [1],
          [1],
          [0]]

In [ ]:
features

<tf.Tensor: id=424, shape=(4, 2), dtype=float32, numpy=
array([[0., 0.],
       [0., 1.],
       [1., 0.],
       [1., 1.]], dtype=float32)>

In [ ]:
dataset  = tf.data.Dataset.from_tensor_slices((X_data,y_data)).batch(len(X_data))

In [ ]:
def preprocess_data(features,labels):
  features  = tf.cast(features,tf.float32)
  labels = tf.cast(labels, tf.float32)
  return features, labels

In [ ]:
W1 = tf.Variable(tf.random_normal([2,1]),name='weight1')
b1 = tf.Variable(tf.random_normal([1]),name='bias1')

W2 = tf.Variable(tf.random_normal([2,1]),name='weight2')
b2 = tf.Variable(tf.random_normal([1]),name='bias2')

W3 = tf.Variable(tf.random_normal([2,1]),name='weight3')
b3 = tf.Variable(tf.random_normal([1]),name='bias3')

In [ ]:
def neural_net(features):
  layer1 = tf.sigmoid(tf.matmul(features, W1) +b1)
  layer2 = tf.sigmoid(tf.matmul(features, W2) +b2)
  layer3 = tf.concat([layer1,layer2],-1)
  layer3 = tf.reshape(layer3,shape =[-1,2])
  hypothesis = tf.sigmoid(tf.matmul(layer3,W3) +b3)
  return hypothesis

def loss_fn(hypothesis,labels):
  cost = -tf.reduce_mean(labels*tf.log(hypothesis)+(1-labels)*tf.log(1-hypothesis))
  return cost

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)

def accuracy_fn(hypothesis,labels):
  predicted =  tf.cast(hypothesis > 0.5,dtype=tf.float32)
  accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted,labels),dtype=tf.float32))
  return accuracy

def grad(features,lables):
  with tf.GradientTape() as tape:
    loss_value = loss_fn(neural_net(features),features,labels)
    return tape.gradient(loss_value,[W1,W2,W3,b1,b2,b3])

EPOCHES = 50000

for step in range(EPOCHES):
  for features, labels in tfe.Iterator(dataset):
      features, labels = preprocess_data(features,labels)
      grads = grad(neural_net(features),labels)
      optimizer.apply_gradients(grades_and_vars=zip(grads,[W1,W2,W3,b1,b2,b3]))
      if step % 5000 == 0:
        print("Iter: {},loss:{:.4f}".format(step,loss_fn(nerual_net(features),labels)))
X_data, y_data = preprocess_data(X_data,y_data)
test_acc = accuracy_fn(neural_net(X_data),y_data)
print("testset Accuracy:{:.4f".format(test_acc))

#Relu

In [37]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os

In [38]:
def normalize(train_data,test_data):
  train_data = train_data.astype(np.float32)/255.0
  test_data = test_data.astype(np.float32)/255.0
  return train_data,test_data

def load_mnist():
  (train_data,train_labels),(test_data,test_labels) = mnist.load_data()
  train_data = np.expand_dims(train_data,axis=-1)
  test_data = np.expand_dims(test_data,axis=-1)
  train_data,test_data = normalize(train_data,test_data)
  train_labels = to_categorical(train_labels,10)
  test_labels = to_categorical(test_labels,10)

  return train_data,train_labels,test_data,test_labels

def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

In [55]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.RandomNormal()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(2):
            self.model.add(dense(256, weight_init))
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):
        x = self.model(x)

        return x

In [60]:
#Create network
#네트워크를 짤때 어떠한 함수를 이용할것인지 먼저 생각해야함

#1.shape을 펼쳐주는 역할
def flatten():
  return tf.keras.layers.Flatten()

#2.Dense layer를 사용할 것이기 때문에 kerase에 Dense를 만들어줌
def dense(channel,weight_init):
  return tf.keras.layers.Dense(units=channel,use_bias=True,kernel_initializer=weight_init)

def relu():
  return tf.keras.layers.Activation(tf.keras.activations.relu)

#tf.keras.model을 상속해야 class 모델을 만들 수 있음
class create_model(tf.keras.Model):
  def __init__(self,label_dim):
    super(create_model,self).__init__()

    weight_init = tf.keras.initializers.RandomNormal()
    self.model = tf.keras.Sequential()
    self.model.add(flatten()) # [N,28,28,1] -> [N,784]

    for i in range(2):
      # [N,784] -> [N,256] -> [N, 256]
      self.model.add(dense(256,weight_init))
      self.model.add(relu())

    self.model.add(dense(label_dim,weight_init)) # [N,256] -> [N,10]

    def call(self,x,training=None,mask=None):
      x=self.model(x)

      return x

#model,image,lable을 받음
def loss_fn(model,images,labels):
  #model에 image를 넣고, logit을 구함
  logits = model(images,training=True)
  #model에서 구한 softmax(logit)로 예측한 값과 lables의 차이를 loss를 구함
  loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
  return loss

def accuracy_fn(model,images,labels):
  logits = model(images,training=False)
  #argmax는 logit과 label에서 가장 숫자가 큰 값의 위치를 알려주는 함수임 logit과 lable은 다음과 같이 이루어져 있음 -1은 lable_dim을 의미함. [batch size,label_dim]
  prediction = tf.equal(tf.argmax(logits,-1),tf.argmax(labels,-1))
  #accuracy를 구하는것은 boolen보다 숫자값으로 바꿔야 계산이 되기 때문에 바꾸어서 저장
  accuracy = tf.reduce_mean(tf.cast(prediction,tf.float32))
  return accuracy

def grad(model,images,labels):
  with tf.GradientTape() as tape:
    loss = loss_fn(model,images,labels)
  return tape.gradient(loss,model.variables)

In [40]:
#하이퍼 파라미터 설정

train_x,train_y,test_x,test_y = load_mnist()

In [51]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

#데이터가 6만장을 네트워크에 넣기에는 메모리에 부담을 주기 때문에, 배치사이즈 만큼 줘야 함 아래는, 그 배치를 어떻게 네트워크에 던져주느냐에 관한 이야기
#shuffle 데이터셋을 잘 썩기,buffer_size는 데이터셋보다 커야함
#prefetch 미리 메모리에 해당 배치사이즈 만큼 올려두어서 빠르게 학습 시키는 방법
#batch   몇개 만큼 네트워크에 던져주는지?
""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

In [56]:
""" Model """
network = create_model_class(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_softmax'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

In [61]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):            
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1                
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.6837, train_loss: 2.15999317, train_accuracy: 0.3828, test_Accuracy: 0.2937
Epoch: [ 0] [    1/  468] time: 1.0490, train_loss: 2.13213396, train_accuracy: 0.5156, test_Accuracy: 0.4757
Epoch: [ 0] [    2/  468] time: 1.4091, train_loss: 2.02703667, train_accuracy: 0.5703, test_Accuracy: 0.5426
Epoch: [ 0] [    3/  468] time: 1.6543, train_loss: 1.98404002, train_accuracy: 0.5859, test_Accuracy: 0.6182
Epoch: [ 0] [    4/  468] time: 1.9149, train_loss: 1.90166497, train_accuracy: 0.6250, test_Accuracy: 0.6411
Epoch: [ 0] [    5/  468] time: 2.1568, train_loss: 1.84366655, train_accuracy: 0.6406, test_Accuracy: 0.6650
Epoch: [ 0] [    6/  468] time: 2.5194, train_loss: 1.71306193, train_accuracy: 0.6641, test_Accuracy: 0.7028
Epoch: [ 0] [    7/  468] time: 2.7617, train_loss: 1.56476462, train_accuracy: 0.7578, test_Accuracy: 0.7177
Epoch: [ 0] [    8/  468] time: 3.1305,

#Weight Initialization
- 실제 로스 그래프는 굉장히 복잡해서 local minima로 학습을 하게 된다면, global minima보다 성능이 좋지 않을 것임
- 따라서 weight 초기화는 네트워크가 어떤 출발지점에서 loss를 찾아가는 것인지 정하는 것이며, Xavier는 이런 것을 잘 정하는 것을 도와주는 방법중 하나임. 
- He initialization은 Relu함수에 사용하면 좋은 웨이트 초기화법임. 분산은 자비어에 보다 2배 더 큼

In [71]:
class create_model_class_Xavier(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class_Xavier, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(2):
            self.model.add(dense(256, weight_init))
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):
        x = self.model(x)

        return x

In [72]:
""" Model """
network2 = create_model_class_Xavier(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_softmax'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

In [73]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network2)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network2, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):            
                grads = grad(network2, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network2.variables))

                train_loss = loss_fn(network2, train_input, train_label)
                train_accuracy = accuracy_fn(network2, train_input, train_label)
                
                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network2, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1                
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network2, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Success to read nn_softmax-468-1
 [*] Load SUCCESS
